In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate as sci
from scipy.optimize import fsolve
import Ch8_functions as Ch8

In [2]:
#Μέρος 1

In [3]:
def Bonds(YTM, CpnRate, CpnPeriod, Maturity, FV, Amortized = False, PrincipalPaymentSchedule = np.empty(1), TimeFromMaturity = np.empty(1)):
    
    
    if (Maturity >= CpnPeriod):
        
        CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
        SpotTenors = np.flip(CashflowTimes)
        SpotRates=np.zeros_like(SpotTenors)+YTM
        SpotCompounding = np.empty_like(SpotTenors, dtype=object); 
        SpotCompounding[:] = 1/CpnPeriod

        Price_Spot, Price_fwd, Cfls_df = Ch8.BondPriceOffSpotCurve(SpotRates,SpotTenors,SpotCompounding, 
                                        CpnRate,CpnPeriod, Maturity, 0, FV, Amortized, PrincipalPaymentSchedule, TimeFromMaturity)
        s1 = (Cfls_df['TotalCfls']*Cfls_df['df(0,t)'])
        s2 = s1*Cfls_df['CflTimes']
        McCauley_Duration = sum(s2)/Price_Spot

        Modified_Duration = McCauley_Duration/(1 + YTM*CpnPeriod)
        
        Dollar_Duration = Modified_Duration*Price_Spot

        Basic_Point_Value = Dollar_Duration/(10**4)
    else:
        Price_Spot = FV*(1 + CpnRate)/(1 + YTM*Maturity)
        McCauley_Duration = Maturity
        Modified_Duration = McCauley_Duration/(1 + YTM)
        Dollar_Duration = Modified_Duration * Price_Spot
        Basic_Point_Value = Dollar_Duration/(10**4)
    
    return Dollar_Duration,  Basic_Point_Value, Modified_Duration, McCauley_Duration, Price_Spot

##δημιουργω μια συναρτηση που υπολογίζει τα μέτρα ευαισθησίας
##για δύο περιπτώσεις, maturity >= CpnPeriod maturity < CpnPeriod

In [4]:
##Παράδειγμα 

CpnRate = 0.03
CpnPeriod = 1
FV = 100
Maturity = 0.78
YTM = 0.0334
Amortized = False

In [5]:
Dollar_Duration,  Basic_Point_Value, Modified_Duration, McCauley_Duration, Price_Spot= Bonds(YTM, CpnRate, CpnPeriod, Maturity, FV, Amortized, PrincipalPaymentSchedule = np.empty(1), TimeFromMaturity = np.empty(1))
Dollar_Duration,  Basic_Point_Value, Modified_Duration, McCauley_Duration, Price_Spot

(75.76942630139004,
 0.007576942630139004,
 0.7547900135475131,
 0.78,
 100.3847758203288)

In [6]:
#Μέρος 2

In [7]:
def Bonds2(Position, data):

    data['buy'] = Position
    pps = []
    tms = []
    for row in range(len(data)):
        if (data.loc[row, "Amortized"] == True):
            tms = np.array([int(x) for x in data["TimeFromMaturity"][row].split(", ")])
            pps = np.array([int(x) for x in data["PrincipalPaymentSchedule"][row].split(", ")])##χωρίζω τα δεδομένα των οποίων τα ομόλογα αποσβένονται
        else:
            tms = np.empty(1)
            pps = np.empty(1)

        dd, bpv, md, mcc, ps = Bonds(YTM = data['YTM'][row], CpnRate = data['CpnRate'][row], CpnPeriod = 1/(data['CpnFreq'][row]), Maturity = data['Maturity'][row], FV = data['FV'][row], Amortized = data['Amortized'][row], PrincipalPaymentSchedule = pps, TimeFromMaturity = tms)
        data.loc[row, 'Dollar Duration'] = dd

        data.loc[row, 'Modified Duration'] = md
        data.loc[row, 'McCaley Duration'] = mcc
        data.loc[row, 'weights'] = ps##αποθηκεύω τις αξίες του κάθε ομολόγου
        
    data['weights'] = (data['weights']*data['buy'])/sum(data['buy']*data['weights'])##φτιάχνω την βαρύτητα των ομολόγων που θα αγοράσω
    data['Dollar Duration'] = data['Dollar Duration']*data['buy']

    data['Modified Duration'] = data['Modified Duration']*data['weights']
    data['McCaley Duration'] = data['McCaley Duration']*data['weights']
    
    data2 = pd.DataFrame()
    data2.loc[0, 'Dollar Duration'] = sum(data['Dollar Duration'])
    data2.loc[0, 'Basic Point Value'] = data2.loc[0, 'Dollar Duration']*10**(-4)
    data2.loc[0, 'Modified Duration'] = sum(data['Modified Duration'])
    data2.loc[0, 'McCaley Duration'] = sum(data['McCaley Duration'])
    return data2

##δημιουργώ μια δεύτερη συνάρτηση όπου καλώ την πρώτη. Μέσα στην επανάληψη καλώ την πρώτη συνάρτηση για κάθε ομόλογο
##και βρίσκω τα μέτα ευαισθησίας για το καθένα. Έπειτα, βρίσκω τα συνολικά μέτρα ευαισθησίας για όλο το χαρτοφυλάκιο
##πολλαπλασιάζοντας το Dollar Duration με τα μερίδια που θα αγοράσω από το καθένα και Modified Duration McCalay Duration 
##με την βαρύτητα των ομολόγων

In [8]:
#Μέρος 3

BondPortfolio = pd.read_excel('BondPortfolio2Manage.xlsx')
BondPortfolio

,Bonds,CpnRate,Maturity,FV,YTM,CpnFreq,CleanPrices,Amortized,PrincipalPaymentSchedule,TimeFromMaturity
0,1,0.0300,0.78,100,0.033423,1,99.723043,False,NaN,NaN
1,2,0.0400,1.30,200,0.040984,1,99.860241,False,NaN,NaN
2,3,0.0500,2.00,250,0.048088,1,100.356465,False,NaN,NaN
3,4,0.0200,3.05,300,0.053205,2,90.762891,False,NaN,NaN
4,5,0.0250,4.12,210,0.054640,2,89.193009,False,NaN,NaN
5,6,0.0400,4.67,300,0.054988,1,93.946283,False,NaN,NaN
6,7,0.0375,5.14,120,0.053747,2,92.782108,False,NaN,NaN
7,8,0.0450,5.56,80,0.053086,2,96.147414,False,NaN,NaN
8,9,0.0395,6.62,230,0.050985,4,93.579870,False,NaN,NaN
9,10,0.0450,7.53,160,0.050106,2,96.828165,False,NaN,NaN


In [9]:
BondPortfolio = pd.DataFrame(data = BondPortfolio)
BondPortfolio

##μετατρέπω το εξελ σε DataFrame

,Bonds,CpnRate,Maturity,FV,YTM,CpnFreq,CleanPrices,Amortized,PrincipalPaymentSchedule,TimeFromMaturity
0,1,0.0300,0.78,100,0.033423,1,99.723043,False,NaN,NaN
1,2,0.0400,1.30,200,0.040984,1,99.860241,False,NaN,NaN
2,3,0.0500,2.00,250,0.048088,1,100.356465,False,NaN,NaN
3,4,0.0200,3.05,300,0.053205,2,90.762891,False,NaN,NaN
4,5,0.0250,4.12,210,0.054640,2,89.193009,False,NaN,NaN
5,6,0.0400,4.67,300,0.054988,1,93.946283,False,NaN,NaN
6,7,0.0375,5.14,120,0.053747,2,92.782108,False,NaN,NaN
7,8,0.0450,5.56,80,0.053086,2,96.147414,False,NaN,NaN
8,9,0.0395,6.62,230,0.050985,4,93.579870,False,NaN,NaN
9,10,0.0450,7.53,160,0.050106,2,96.828165,False,NaN,NaN


In [10]:
BondPortfolio = BondPortfolio.drop(['Bonds'], axis = 1)
BondPortfolio = BondPortfolio.drop(['CleanPrices'], axis = 1)
BondPortfolio 

##βγάζω τις στήλες όπου δεν χρειάζομαι

,CpnRate,Maturity,FV,YTM,CpnFreq,Amortized,PrincipalPaymentSchedule,TimeFromMaturity
0,0.0300,0.78,100,0.033423,1,False,NaN,NaN
1,0.0400,1.30,200,0.040984,1,False,NaN,NaN
2,0.0500,2.00,250,0.048088,1,False,NaN,NaN
3,0.0200,3.05,300,0.053205,2,False,NaN,NaN
4,0.0250,4.12,210,0.054640,2,False,NaN,NaN
5,0.0400,4.67,300,0.054988,1,False,NaN,NaN
6,0.0375,5.14,120,0.053747,2,False,NaN,NaN
7,0.0450,5.56,80,0.053086,2,False,NaN,NaN
8,0.0395,6.62,230,0.050985,4,False,NaN,NaN
9,0.0450,7.53,160,0.050106,2,False,NaN,NaN


In [11]:
Position = (100/BondPortfolio['FV'])

BondPortfolio = BondPortfolio.fillna(0)
BondPortfolio

##βρίσκω τα μερίδια από κάθε ομόλογο

,CpnRate,Maturity,FV,YTM,CpnFreq,Amortized,PrincipalPaymentSchedule,TimeFromMaturity
0,0.0300,0.78,100,0.033423,1,False,0,0
1,0.0400,1.30,200,0.040984,1,False,0,0
2,0.0500,2.00,250,0.048088,1,False,0,0
3,0.0200,3.05,300,0.053205,2,False,0,0
4,0.0250,4.12,210,0.054640,2,False,0,0
5,0.0400,4.67,300,0.054988,1,False,0,0
6,0.0375,5.14,120,0.053747,2,False,0,0
7,0.0450,5.56,80,0.053086,2,False,0,0
8,0.0395,6.62,230,0.050985,4,False,0,0
9,0.0450,7.53,160,0.050106,2,False,0,0


In [12]:
Position

0     1.000000
1     0.500000
2     0.400000
3     0.333333
4     0.476190
5     0.333333
6     0.833333
7     1.250000
8     0.434783
9     0.625000
10    1.250000
11    1.000000
Name: FV, dtype: float64

In [13]:
TotalBondPortfolio = pd.DataFrame()
TotalBondPortfolio = Bonds2(Position, BondPortfolio)
TotalBondPortfolio

##βρίσκω τα μέτρα ευαισθησίας του χαρτοφυλακίου

,Dollar Duration,Basic Point Value,Modified Duration,McCaley Duration
0,4627.651123,0.462765,4.026075,4.167108


In [14]:
#Μέρος 4

In [15]:
CpnRate = 0
Maturity = 6
CpnPeriod = 0.5
YTM = 0.05
FV = 100

In [16]:
# NoDollarDuration, NoBasicPointValue, NoModifiedValue, NoMcCaleyDuration, NoPriceSpot = Bonds(YTM, CpnRate, CpnPeriod, Maturity, FV, Amortized = False, PrincipalPaymentSchedule = np.empty(1), TimeFromMaturity = np.empty(1))
# NoDollarDuration, NoBasicPointValue, NoModifiedValue, NoMcCaleyDuration, NoPriceSpot

NoDollarDuration = (Maturity/(1 + YTM*CpnPeriod))*FV*(1/((1 + YTM*CpnPeriod)**((1/CpnPeriod)*Maturity)))


##2 τρόποι να βρούμε το πόσα μερίδια για να αντισταθμίσουμε τον κίνδυνο

In [17]:
TotalBondPortfolio['Dollar Duration']/NoDollarDuration


0    10.632114
Name: Dollar Duration, dtype: float64

In [18]:
# τα μερίδια είναι 10.63 περίπου 11 δηλαδή που πρέπει να πουλήσουμε